# 📚 Books 情绪分析流程总结

1. **数据准备**
   - 读取 `books_with_categories.csv`，包含书籍元数据（isbn13, title, description 等）。

2. **构建情绪分类器**
   - 使用 Hugging Face 模型：`j-hartmann/emotion-english-distilroberta-base`
   - 输出 7 类情绪：anger, disgust, fear, joy, sadness, surprise, neutral

3. **句子级预测**
   - 将书籍简介 `description` 按句号切分 → `classifier(sentences)` → 得到逐句情绪分布。

4. **图书级特征提取**
   - 对每本书的所有句子取 **各情绪的最大值** → 生成情绪向量。

5. **批量处理与合并**
   - 遍历所有书籍，生成情绪 DataFrame → 与原始 books 数据按 `isbn13` 合并。

6. **保存结果**
   - 导出为 `books_with_emotions.csv`，每本书多出 7 个情绪特征列。

✅ 最终结果：
每本书拥有 **原始信息 + 情绪特征向量**，可用于推荐、补全分类或分析。


In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [7]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "mps")
classifier("I love this!")
#不同硬件（CPU vs GPU vs MPS）在浮点运算上会有 非确定性差异，即使输入相同，输出的概率也可能略有不同。

Device set to use mps


[[{'label': 'surprise', 'score': 0.48696979880332947},
  {'label': 'neutral', 'score': 0.2234414517879486},
  {'label': 'joy', 'score': 0.14913064241409302},
  {'label': 'anger', 'score': 0.07174026966094971},
  {'label': 'sadness', 'score': 0.04664652422070503},
  {'label': 'disgust', 'score': 0.01629774644970894},
  {'label': 'fear', 'score': 0.005773560609668493}]]

In [8]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [9]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.16985200345516205},
  {'label': 'sadness', 'score': 0.11640875786542892},
  {'label': 'surprise', 'score': 0.02070065401494503},
  {'label': 'disgust', 'score': 0.019100766628980637},
  {'label': 'joy', 'score': 0.015161258168518543},
  {'label': 'anger', 'score': 0.003935154993087053}]]

In [10]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.1403856724500656},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.047942448407411575},
  {'label': 'anger', 'score': 0.009156345389783382},
  {'label': 'disgust', 'score': 0.0026284728664904833},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937166571617126},
  {'label': 'disgust', 'score': 0.2735905349254608},
  {'label': 'joy', 'score': 0.10908280313014984},
  {'label': 'sadness', 'score': 0.09362749755382538},
  {'label': 'anger', 'score': 0.040478236973285675},
  {'label': 'surprise', 'score': 0.02697022631764412},
  {'label': 'fear', 'score': 0.006879068911075592}],
 [{'label': 'neutral', 'score': 0.6462157964706421},
  {'label': 'sadness', 'score': 0.24273350834846497},
  {'label': 'disgust', 'score': 0.0434226468205452},
  {'label': 'surprise', 'score': 0.028300516307353973},
  {'label': 'joy', 'score': 0.01421145

In [11]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)#我们的模型classifier

In [12]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [13]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.1403856724500656},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.047942448407411575},
 {'label': 'anger', 'score': 0.009156345389783382},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [14]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [15]:
predictions[3]

[{'label': 'fear', 'score': 0.9281684160232544},
 {'label': 'anger', 'score': 0.03219081833958626},
 {'label': 'neutral', 'score': 0.01280867401510477},
 {'label': 'sadness', 'score': 0.008756855502724648},
 {'label': 'surprise', 'score': 0.008597873151302338},
 {'label': 'disgust', 'score': 0.008431804366409779},
 {'label': 'joy', 'score': 0.0010455820010975003}]

In [16]:
predictions

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.1403856724500656},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.047942448407411575},
  {'label': 'anger', 'score': 0.009156345389783382},
  {'label': 'disgust', 'score': 0.0026284728664904833},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937166571617126},
  {'label': 'disgust', 'score': 0.2735905349254608},
  {'label': 'joy', 'score': 0.10908280313014984},
  {'label': 'sadness', 'score': 0.09362749755382538},
  {'label': 'anger', 'score': 0.040478236973285675},
  {'label': 'surprise', 'score': 0.02697022631764412},
  {'label': 'fear', 'score': 0.006879068911075592}],
 [{'label': 'neutral', 'score': 0.6462157964706421},
  {'label': 'sadness', 'score': 0.24273350834846497},
  {'label': 'disgust', 'score': 0.0434226468205452},
  {'label': 'surprise', 'score': 0.028300516307353973},
  {'label': 'joy', 'score': 0.01421145

In [17]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156345389783382},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.047942448407411575},
 {'label': 'neutral', 'score': 0.1403856724500656},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296027541160583}]

In [19]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}
#对每个情绪，取这个书所有句子里的 最高分，作为该书在该情绪下的“代表性强度”。

In [20]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])
#批量处理前 10 本书的简介，把每本书的情绪得分算出来

In [21]:
emotion_scores

{'anger': [np.float64(0.051973048597574234),
  np.float64(0.6126187443733215),
  np.float64(0.051973048597574234),
  np.float64(0.3514830768108368),
  np.float64(0.08141232281923294),
  np.float64(0.23222492635250092),
  np.float64(0.5381841063499451),
  np.float64(0.051973048597574234),
  np.float64(0.300670862197876),
  np.float64(0.051973048597574234)],
 'disgust': [np.float64(0.2735905349254608),
  np.float64(0.348285436630249),
  np.float64(0.15766695141792297),
  np.float64(0.15766695141792297),
  np.float64(0.18449515104293823),
  np.float64(0.7271747589111328),
  np.float64(0.15766695141792297),
  np.float64(0.15766695141792297),
  np.float64(0.27948087453842163),
  np.float64(0.17792636156082153)],
 'fear': [np.float64(0.9281684160232544),
  np.float64(0.942527711391449),
  np.float64(0.9723208546638489),
  np.float64(0.3607070744037628),
  np.float64(0.09504328668117523),
  np.float64(0.038786713033914566),
  np.float64(0.747427761554718),
  np.float64(0.4044956862926483),
  

In [22]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:31<00:00, 34.22it/s]


全部书


对一本书的所有句子，找出每种情绪的 最大得分。

In [24]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [25]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.051973,0.273591,0.928168,0.932798,0.646216,0.967157,0.729603,9780002005883
1,0.612619,0.348285,0.942528,0.704421,0.887940,0.074825,0.252545,9780002261982
2,0.051973,0.157667,0.972321,0.767237,0.608933,0.074825,0.046931,9780006178736
3,0.351483,0.157667,0.360707,0.251881,0.732687,0.074825,0.046931,9780006280897
4,0.081412,0.184495,0.095043,0.035207,0.925904,0.475881,0.046931,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656,9788172235222
5193,0.057145,0.157667,0.038787,0.400263,0.883199,0.074825,0.227765,9788173031014
5194,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625,9788179921623
5195,0.051973,0.157667,0.459269,0.759455,0.951104,0.368110,0.067808,9788185300535


In [26]:
books = pd.merge(books, emotions_df, on = "isbn13")

In [27]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.051973,0.273591,0.928168,0.932798,0.646216,0.967157,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704421,0.887940,0.074825,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.051973,0.157667,0.972321,0.767237,0.608933,0.074825,0.046931
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.157667,0.360707,0.251881,0.732687,0.074825,0.046931
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.035207,0.925904,0.475881,0.046931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.057145,0.157667,0.038787,0.400263,0.883199,0.074825,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.051973,0.157667,0.459269,0.759455,0.951104,0.368110,0.067808


In [28]:
books.to_csv("books_with_emotions.csv", index = False)

再把它和 books 合并，变成一个更丰富的表格：
👉 原始书籍信息 + 每本书的情绪特征。